#### Ai Agents in LangGraph Course

In [1]:
import os
import re
from pathlib import Path
from dotenv import load_dotenv

from langchain_groq import ChatGroq

# Get environment variables
dotenv_path = Path('./.env')
load_dotenv(dotenv_path=dotenv_path)

os.environ["NEO4J_URI"] = os.getenv('uri')
os.environ["NEO4J_USERNAME"] = os.getenv('user_name')
os.environ["NEO4J_PASSWORD"] = os.getenv('password')
os.environ["GROQ_API_KEY"] = os.getenv('GROQ_API_KEY')
os.environ["TAVILY_API_KEY"] = os.getenv('TAVILY_API_KEY')

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated # to construct the agent's state
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_community.tools.tavily_search import TavilySearchResults

In [3]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add] # operator.add allows us to add messages instead of replacing them when the LLM's output is returned

In [4]:
# # Used for persistence
# from langgraph.checkpoint.sqlite import SqliteSaver
# memory = SqliteSaver.from_conn_string(":memory:")

In [5]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState) # initialize graph

        # Add nodes
        graph.add_node("llm", self.call_groq)
        graph.add_node("action", self.take_action)
        
        # The edge where the decision to use a tool is made
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        
        # Create edge and set starting point of the graph
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")

        self.graph = graph.compile() # Build graph
        
        self.tools = {t.name: t for t in tools} # Save the tools' names that can be used
        self.model = model.bind_tools(tools) # Provide the name of the tools to the agent

    # Tells the agent if action is needed by checking the last message in the state which is supposed to contain this info
    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    # Call the LLM and update the Agent's State by adding the response
    def call_groq(self, state: AgentState):
        messages = state['messages']
        if self.system: messages = [SystemMessage(content=self.system)] + messages
        
        message = self.model.invoke(messages)
        return {'messages': [message]}

    # Search for the tool and use it
    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check if tool name is found in list of tools 
                print("\n ....tool name not found in list of tools....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args']) 

            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))

        print("Back to the model!")
        return {'messages': results}

In [6]:
from langchain.graphs import Neo4jGraph
from langchain_core.tools import tool

# Connect to graph
graph = Neo4jGraph()

@tool
def query_graph(query):
  """Requires get_graph_schema to be run before using this function. This function is to Query from Neo4j knowledge graph using Cypher."""
  return graph.query(query)

In [7]:
task = "Task: You will use a Neo4j database to improve your answer. You will query the possible occupation titles that are suitable for my character."

schema_context = f"Here is the graph's schema: {graph.structured_schema}."

property_values = f"Property Values: empty"

query_approach = "Querying approach: You will not use 'LIMIT'. If Property Values: empty, you will not use general queries and will not include 'WHERE' or try to specify property values inside your Cypher code."

output = "Your final output: Interpret all the queried data, choose up to 15 suitable careers for me, list them in bullet points and include a brief explanation of how each path suites my personality. Include Cypher code in your answer."

tone = "Output's tone: Make your output friendly, fun and easy to read."

personal_info = "Personal Info: I love people and I am a good listener. I enjoy observation and analysis. I prefer being with abults rather than with kids and I also have computer programming skills."

reminder = "Reminder: If Property Values: empty, you will not use 'WHERE' or try to specify property values inside your Cypher code. Under no circumstances should you use 'DELETE'. Find the occupations that suite my character."

prompt = f"{task}\ {schema_context}\ {property_values}\ {query_approach}\ {output}\ {tone}\ {personal_info}\ {reminder}"

In [10]:
# Prepare agent and the initial prompt
model = ChatGroq(temperature=0, groq_api_key=os.environ["GROQ_API_KEY"], model_name="llama-3.1-70b-versatile")
abot = Agent(model, [query_graph], system=prompt)

# Prepare Human message
query = "I am interested in mental disorders, I have previous experience in teachin kids and I like kids."
messages = [HumanMessage(content=query)]

In [11]:
# Give the agent the messsage
result = abot.graph.invoke({"messages": messages})
print(result['messages'][-1].content)


 In Exists Action => content='' additional_kwargs={'tool_calls': [{'id': 'call_bwq8', 'function': {'arguments': '{"query": "MATCH (n:Occupation)-[:need_for_personality_trait]->(m:Personality_Trait) RETURN n.title, m.title"}', 'name': 'query_graph'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 583, 'total_tokens': 624, 'completion_time': 0.164, 'prompt_time': 0.137492243, 'queue_time': 0.005134392000000015, 'total_time': 0.301492243}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_5c5d1b5cfb', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-1bc16258-8fe0-4b40-aa5b-236383f94e68-0' tool_calls=[{'name': 'query_graph', 'args': {'query': 'MATCH (n:Occupation)-[:need_for_personality_trait]->(m:Personality_Trait) RETURN n.title, m.title'}, 'id': 'call_bwq8', 'type': 'tool_call'}] usage_metadata={'input_tokens': 583, 'output_tokens': 41, 'total_tokens': 624}

 take_action => [{'name': 'query_graph', 'args': {'q

In [ ]:
# ## Visualize Graph
# from IPython.display import Image

# Image(abot.graph.get_graph().draw_png())

### Persistence